In [2]:
import pandas as pd
import numpy as np

In [5]:
df = pd.read_parquet('../api/data/solar.parquet')
df.head()

,Customer,Consumption Category,date,0:30,1:00,1:30,2:00,2:30,3:00,3:30,...,19:30,20:00,20:30,21:00,21:30,22:00,22:30,23:00,23:30,0:00
0,1,CL,1/07/2012,1.250,1.250,1.250,1.263,0.131,0.569,0.556,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.081
1,1,GC,1/07/2012,0.855,0.786,0.604,0.544,0.597,0.612,1.245,...,0.329,0.374,0.447,0.549,0.136,0.288,0.181,0.651,0.090,0.068
2,1,GG,1/07/2012,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
3,1,CL,2/07/2012,1.250,1.250,1.125,0.000,0.925,0.231,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.069
4,1,GC,2/07/2012,0.309,0.082,0.059,0.097,0.290,0.155,0.703,...,0.696,0.353,0.464,0.229,0.811,0.222,0.306,1.034,0.136,0.067


In [10]:
dfmod = df.melt(id_vars=['Customer', 'Consumption Category', 'date'], value_name='amount', var_name='hour')

df2 = dfmod.copy()
df2['Datetime'] = pd.to_datetime(df2['date'] + df2['hour'], format='%d/%m/%Y%H:%M')
df2.drop(columns=['date', 'hour'], inplace=True)
df2.sort_values('Datetime', inplace=True)
df2.set_index('Datetime', inplace=True)

In [ ]:
dfmod.to_parquet('melt.parquet')

df2.to_parquet('melt_datetime.parquet')

In [11]:
df2.head()

,Customer,Consumption Category,amount
Datetime,,,
2012-07-01,102,GC,0.113
2012-07-01,230,GC,0.344
2012-07-01,230,GG,0.000
2012-07-01,88,GG,0.000
2012-07-01,88,GC,0.142


In [8]:
dfmod.head()

,Customer,Consumption Category,date,hour,amount
0,1,CL,1/07/2012,0:30,1.250
1,1,GC,1/07/2012,0:30,0.855
2,1,GG,1/07/2012,0:30,0.000
3,1,CL,2/07/2012,0:30,1.250
4,1,GC,2/07/2012,0:30,0.309
